### Importações

In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import os
import sys

from sklearn.neighbors import LocalOutlierFactor
from sklearn.preprocessing import MinMaxScaler

In [ ]:
%load_ext autoreload
%autoreload 2
sys.path.append(os.path.abspath('..'))
from functions.clean_df import load_and_combine_csvs,clean_dataframe,add_confidential_flags
from functions.state_imput import apply_state_estimation

### Criar o DF

In [ ]:
raw_path = '../raw_data/'
df_raw = load_and_combine_csvs(raw_path)

df_clean = clean_dataframe(df_raw)

df_clean = add_confidential_flags(df_clean)
df_clean = apply_state_estimation(df_clean)

df_clean.info()

### Feature Engeniering necessárias para o LOF

In [ ]:
## Log do valor da transação
df_clean['LOG_VALOR'] = np.log1p(df_clean['VALOR TRANSAÇÃO'])

## Frequência das categorias
cols_categoria = ['NOME ÓRGÃO', 'ESTADO_ESTIMADO', 'NOME FAVORECIDO']

for col in cols_categoria:
    freq_map = df_clean[col].value_counts(normalize=True)
    df_clean[f'FREQ_{col}'] = df_clean[col].map(freq_map)

In [ ]:
# Criar coluna de Semestre (1 ou 2)
df_clean['SEMESTRE_EXTRATO'] = np.where(df_clean['MÊS EXTRATO'] <= 6, 1, 2)

# Média por Órgão no MÊS
df_clean['MEDIA_VALOR_ORGAO_MES'] = df_clean.groupby(['NOME ÓRGÃO', 'ANO EXTRATO', 'MÊS EXTRATO'])['VALOR TRANSAÇÃO'].transform('mean')

# Média por Órgão no SEMESTRE
df_clean['MEDIA_VALOR_ORGAO_SEM'] = df_clean.groupby(['NOME ÓRGÃO', 'ANO EXTRATO', 'SEMESTRE_EXTRATO'])['VALOR TRANSAÇÃO'].transform('mean')

# Razão do valor da transação pela média do órgão no período
df_clean['RATIO_MES'] = df_clean['VALOR TRANSAÇÃO'] / df_clean['MEDIA_VALOR_ORGAO_MES']
df_clean['RATIO_SEM'] = df_clean['VALOR TRANSAÇÃO'] / df_clean['MEDIA_VALOR_ORGAO_SEM']

In [ ]:
df_clean.head()

### Selecionando as features

In [ ]:
features_modelo = [
    'LOG_VALOR',
    'RATIO_MES',
    'ANO EXTRATO',
    'MÊS EXTRATO',
    'FREQ_NOME ÓRGÃO',
    'FREQ_ESTADO_ESTIMADO',
    'SIGILOSO'
]

# Criando X
X = df_clean[features_modelo].copy().fillna(0)

scaler = MinMaxScaler()
X_scaled = scaler.fit_transform(X)

### Treinando o modelo

In [ ]:
lof = LocalOutlierFactor(n_neighbors=20, contamination=0.01, n_jobs=-1)

# Fit / Predict
df_clean['LOF_LABEL'] = lof.fit_predict(X_scaled)
df_clean['LOF_SCORE'] = lof.negative_outlier_factor_

### Visualização

In [ ]:
plt.figure(figsize=(14, 7))
sns.scatterplot(
    data=df_clean,
    x='DATA TRANSAÇÃO',
    y='VALOR TRANSAÇÃO',
    hue='LOF_LABEL',
    palette={1: 'lightgrey', -1: 'red'},
    alpha=0.6,
    size='LOF_LABEL',
    sizes=(20, 50)
)
plt.yscale('log')
plt.title('Baseline LOF: Detecção de Anomalias (Vermelho)')
plt.ylabel('Valor da Transação (Log)')
plt.xlabel('Data')
plt.legend(title='Status (1=Ok, -1=Suspeito)')
plt.show()

### Top suspeitos

In [ ]:
cols_view = ['NOME ÓRGÃO', 'NOME FAVORECIDO', 'VALOR TRANSAÇÃO', 'DATA TRANSAÇÃO', 'ESTADO_ESTIMADO', 'LOF_SCORE']
top_anomalias = df_clean[df_clean['LOF_LABEL'] == -1].sort_values('LOF_SCORE', ascending=True).head(10)
top_anomalias.head(10)[cols_view]